In [127]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, date_format, min, mean

import pandas as pd

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1" 
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

spark

In [98]:
attennd = spark.read.csv("attend.csv", header=True, inferSchema=True)
attennd.show()

+------------------------+-----------------------+--------------------------+--------------------+--------------------+-------------+------------+--------------------+-----------------------+
|уникальный номер занятия|уникальный номер группы|уникальный номер участника|       направление 2|       направление 3|онлайн/офлайн|дата занятия|время начала занятия|время окончания занятия|
+------------------------+-----------------------+--------------------------+--------------------+--------------------+-------------+------------+--------------------+-----------------------+
|               401346550|              801346550|                 101352023|   ОНЛАЙН Гимнастика|        ОНЛАЙН Цигун|           Да|  2022-08-01| 2023-05-27 09:00:00|    2023-05-27 10:00:00|
|               401346550|              801346550|                 101385462|   ОНЛАЙН Гимнастика|        ОНЛАЙН Цигун|           Да|  2022-08-01| 2023-05-27 09:00:00|    2023-05-27 10:00:00|
|               401346550|              

In [99]:
users_activity = attennd.drop('уникальный номер занятия', 'уникальный номер группы', 'направление 2', 'направление 3', 'онлайн/офлайн','время окончания занятия')
#del attennd
users_activity.show()

+--------------------------+------------+--------------------+
|уникальный номер участника|дата занятия|время начала занятия|
+--------------------------+------------+--------------------+
|                 101352023|  2022-08-01| 2023-05-27 09:00:00|
|                 101385462|  2022-08-01| 2023-05-27 09:00:00|
|                 101421897|  2022-08-01| 2023-05-27 09:00:00|
|                 101354499|  2022-08-01| 2023-05-27 09:00:00|
|                 101421312|  2022-08-01| 2023-05-27 09:00:00|
|                 101417874|  2022-08-01| 2023-05-27 09:00:00|
|                 101411256|  2022-08-01| 2023-05-27 09:00:00|
|                 101417467|  2022-08-01| 2023-05-27 09:00:00|
|                 101403186|  2022-08-01| 2023-05-27 09:00:00|
|                 101420018|  2022-08-01| 2023-05-27 09:00:00|
|                 101425664|  2022-08-01| 2023-05-27 09:00:00|
|                 101420281|  2022-08-01| 2023-05-27 09:00:00|
|                 101399232|  2022-08-01| 2023-05-27 09

In [107]:
users_first_activities = users_activity.groupBy("уникальный номер участника").agg({"дата занятия":'min'})
users_first_activities = users_first_activities.withColumnRenamed("уникальный номер участника", "уникальный номер")
users_first_activities.show()

+----------------+-----------------+
|уникальный номер|min(дата занятия)|
+----------------+-----------------+
|       101365986|       2022-04-01|
|       101379240|       2022-04-01|
|       101393640|       2022-04-01|
|       101354891|       2022-04-01|
|       101363553|       2022-04-01|
|       101396720|       2022-04-02|
|       101348309|       2022-04-07|
|       101371904|       2022-04-03|
|       101399822|       2022-04-01|
|       101347730|       2022-04-05|
|       101404631|       2022-04-01|
|       101362188|       2022-04-01|
|       101410278|       2022-04-21|
|       101364474|       2022-07-12|
|       101383676|       2022-04-01|
|       101417344|       2022-04-05|
|       101422475|       2022-04-01|
|       101380626|       2022-04-11|
|       101374051|       2022-04-04|
|       101358599|       2022-04-05|
+----------------+-----------------+
only showing top 20 rows



In [104]:
users = spark.read.csv("users.csv", header=True, inferSchema=True)
users.show()

+----------------+--------------------------+-------+-------------+--------------------+
|уникальный номер|дата создание личного дела|    пол|дата рождения|    адрес проживания|
+----------------+--------------------------+-------+-------------+--------------------+
|       101391104|       2019-02-26 15:52:09|Женщина|   1959-09-10|город москва, кон...|
|       101437383|      2022-09-14 13:46:...|Женщина|   1958-06-29|г. москва, больша...|
|       101351338|       2018-03-19 18:34:58|Женщина|   1956-08-28|город москва, лет...|
|       101354578|       2018-03-22 13:19:42|Женщина|   1939-05-19|город москва, лоб...|
|       101411997|      2019-12-17 12:29:...|Женщина|   1951-03-02|г. москва, улица ...|
|       101387414|       2019-01-23 14:14:45|Женщина|   1937-02-17|город москва, сим...|
|       101451078|      2023-03-17 13:57:...|Женщина|   1949-10-08|город москва, ули...|
|       101396634|       2019-04-12 15:48:13|Женщина|   1959-01-28|город москва, мыт...|
|       101406313|   

In [109]:
users = users.join(users_first_activities, on=["уникальный номер"], how="outer")
users.show()

+----------------+--------------------------+-------+-------------+--------------------+-----------------+
|уникальный номер|дата создание личного дела|    пол|дата рождения|    адрес проживания|min(дата занятия)|
+----------------+--------------------------+-------+-------------+--------------------+-----------------+
|       101346552|       2018-03-01 16:58:48|Женщина|   1959-11-16|город москва, зел...|       2022-04-01|
|       101346558|       2018-03-01 18:15:32|Женщина|   1954-04-09|город москва, зел...|       2022-04-01|
|       101346562|       2018-03-01 18:47:34|Женщина|   1946-06-23|город москва, зел...|       2022-04-04|
|       101346581|       2018-03-05 16:07:09|Женщина|   1956-04-19|город москва, зел...|       2022-04-14|
|       101346593|       2018-03-06 14:43:59|Женщина|   1960-04-28|город москва, зел...|       2022-04-04|
|       101346596|       2018-03-07 16:54:00|Женщина|   1943-12-27|город москва, зел...|       2022-04-01|
|       101346597|       2018-03-09 1

In [111]:
users = users.withColumn("min(дата занятия)", to_timestamp("min(дата занятия)"))
users.show()

+----------------+--------------------------+-------+-------------+--------------------+-------------------+
|уникальный номер|дата создание личного дела|    пол|дата рождения|    адрес проживания|  min(дата занятия)|
+----------------+--------------------------+-------+-------------+--------------------+-------------------+
|       101346552|       2018-03-01 16:58:48|Женщина|   1959-11-16|город москва, зел...|2022-04-01 00:00:00|
|       101346558|       2018-03-01 18:15:32|Женщина|   1954-04-09|город москва, зел...|2022-04-01 00:00:00|
|       101346562|       2018-03-01 18:47:34|Женщина|   1946-06-23|город москва, зел...|2022-04-04 00:00:00|
|       101346581|       2018-03-05 16:07:09|Женщина|   1956-04-19|город москва, зел...|2022-04-14 00:00:00|
|       101346593|       2018-03-06 14:43:59|Женщина|   1960-04-28|город москва, зел...|2022-04-04 00:00:00|
|       101346596|       2018-03-07 16:54:00|Женщина|   1943-12-27|город москва, зел...|2022-04-01 00:00:00|
|       101346597| 

In [121]:
users = users.withColumn("дельта регистрации и первого занятия, дней", ((col("min(дата занятия)").cast("long")-col("дата создание личного дела").cast("long"))/86400).cast("int"))
users.show()

+----------------+--------------------------+-------+-------------+--------------------+-------------------+------------------------------------------+
|уникальный номер|дата создание личного дела|    пол|дата рождения|    адрес проживания|  min(дата занятия)|дельта регистрации и первого занятия, дней|
+----------------+--------------------------+-------+-------------+--------------------+-------------------+------------------------------------------+
|       101346552|       2018-03-01 16:58:48|Женщина|   1959-11-16|город москва, зел...|2022-04-01 00:00:00|                                      1491|
|       101346558|       2018-03-01 18:15:32|Женщина|   1954-04-09|город москва, зел...|2022-04-01 00:00:00|                                      1491|
|       101346562|       2018-03-01 18:47:34|Женщина|   1946-06-23|город москва, зел...|2022-04-04 00:00:00|                                      1494|
|       101346581|       2018-03-05 16:07:09|Женщина|   1956-04-19|город москва, зел...|

In [135]:
users = users.fillna(0)
mean_of_deltas = users.select(mean("дельта регистрации и первого занятия, дней")).collect()[0][0]
mean_of_deltas

871.0256047693659

In [138]:
users = users.withColumn('отклоние от медианы дельты', col('дельта регистрации и первого занятия, дней') /mean_of_deltas)
users.show()

+----------------+--------------------------+-------+-------------+--------------------+-------------------+------------------------------------------+--------------------------+
|уникальный номер|дата создание личного дела|    пол|дата рождения|    адрес проживания|  min(дата занятия)|дельта регистрации и первого занятия, дней|отклоние от медианы дельты|
+----------------+--------------------------+-------+-------------+--------------------+-------------------+------------------------------------------+--------------------------+
|       101346552|       2018-03-01 16:58:48|Женщина|   1959-11-16|город москва, зел...|2022-04-01 00:00:00|                                      1491|         1.711775166924965|
|       101346558|       2018-03-01 18:15:32|Женщина|   1954-04-09|город москва, зел...|2022-04-01 00:00:00|                                      1491|         1.711775166924965|
|       101346562|       2018-03-01 18:47:34|Женщина|   1946-06-23|город москва, зел...|2022-04-04 00:00:

In [141]:
newbee_limit_days = 60

In [142]:
users = users.withColumn('is_newbee', col('дельта регистрации и первого занятия, дней') < newbee_limit_days)
users.show()

+----------------+--------------------------+-------+-------------+--------------------+-------------------+------------------------------------------+--------------------------+---------+
|уникальный номер|дата создание личного дела|    пол|дата рождения|    адрес проживания|  min(дата занятия)|дельта регистрации и первого занятия, дней|отклоние от медианы дельты|is_newbee|
+----------------+--------------------------+-------+-------------+--------------------+-------------------+------------------------------------------+--------------------------+---------+
|       101346552|       2018-03-01 16:58:48|Женщина|   1959-11-16|город москва, зел...|2022-04-01 00:00:00|                                      1491|         1.711775166924965|    false|
|       101346558|       2018-03-01 18:15:32|Женщина|   1954-04-09|город москва, зел...|2022-04-01 00:00:00|                                      1491|         1.711775166924965|    false|
|       101346562|       2018-03-01 18:47:34|Женщина|  

In [143]:
users.repartition(1).write.csv("users_is_newbee_feature_added.csv", )

In [148]:
import pandas as pd

df = users.toPandas()
df

,уникальный номер,дата создание личного дела,пол,дата рождения,адрес проживания,min(дата занятия),"дельта регистрации и первого занятия, дней",отклоние от медианы дельты,is_newbee
0,101346552,2018-03-01 16:58:48.000,Женщина,1959-11-16,"город москва, зеленоград, дом 1539",2022-04-01,1491,1.711775,False
1,101346558,2018-03-01 18:15:32.000,Женщина,1954-04-09,"город москва, зеленоград, дом 1820",2022-04-01,1491,1.711775,False
2,101346562,2018-03-01 18:47:34.000,Женщина,1946-06-23,"город москва, зеленоград, дом 1132",2022-04-04,1494,1.715219,False
3,101346581,2018-03-05 16:07:09.000,Женщина,1956-04-19,"город москва, зеленоград, дом 1012",2022-04-14,1500,1.722108,False
4,101346593,2018-03-06 14:43:59.000,Женщина,1960-04-28,"город москва, зеленоград, дом 847",2022-04-04,1489,1.709479,False
...,...,...,...,...,...,...,...,...,...
52329,101451174,2023-03-22 12:38:47.140,Мужчина,1958-07-12,"г. москва, зеленоградская улица, дом 17, корпус 3",NaT,0,0.000000,True
52330,101451200,2023-03-23 16:43:34.515,Женщина,1953-10-03,"город москва, улица гарибальди, дом 11",NaT,0,0.000000,True
52331,101451201,2023-03-23 17:35:38.650,Женщина,1961-12-24,"г. москва, улица молдагуловой, дом 3, корпус 3",NaT,0,0.000000,True
52332,101451204,2023-03-24 10:44:31.834,Женщина,1954-10-04,"город москва, улица симоновский вал, дом 10",NaT,0,0.000000,True


In [149]:
df['is_newbee'].value_counts()

is_newbee
False    40369
True     11965
Name: count, dtype: int64